In [16]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project directory (make sure this path is correct for you)
import os
os.chdir('/content/drive/Othercomputers/My_Mac/sentinel')

# Install required packages
!pip install -q google-cloud-storage wget tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Setup GCS credentials
from google.colab import auth
auth.authenticate_user()

# Set project ID
!gcloud config set project bug-sync-467815

Updated property [core/project].


In [17]:
from google.cloud import storage
from tqdm import tqdm
import os

def upload_to_gcs(local_dir, bucket_name, gcs_prefix=''):
    """Uploads the contents of a directory to a GCS bucket."""
    if not os.path.exists(local_dir):
        print(f"Error: Local directory '{local_dir}' does not exist. Skipping upload.")
        return

    client = storage.Client()
    bucket = client.bucket(bucket_name)
    total_files = sum([len(files) for _, _, files in os.walk(local_dir)])

    with tqdm(total=total_files, desc=f"Uploading from {local_dir}") as pbar:
        for root, dirs, files in os.walk(local_dir):
            for file in files:
                local_path = os.path.join(root, file)
                relative_path = os.path.relpath(local_path, local_dir)
                gcs_path = os.path.join(gcs_prefix, relative_path)

                blob = bucket.blob(gcs_path)
                blob.upload_from_filename(local_path)
                pbar.update(1)

In [ ]:
# --- 1. DOWNLOAD VELODYNE DATA ---
print("--- Processing Velodyne Data (Step 1/2) ---")
velodyne_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_velodyne.zip"
!wget --user-agent="Mozilla/5.0" -O data_odometry_velodyne.zip "{velodyne_url}"

# --- 2. EXTRACT VELODYNE DATA ---
print("\nExtracting velodyne data...")
!unzip -o -q data_odometry_velodyne.zip -d /content/kitti_data
print("Extraction complete.")



--- Processing Velodyne Data (Step 1/2) ---
--2025-10-10 07:33:56--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_velodyne.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.139.209, 3.5.139.170, 52.219.171.65, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.139.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84786535790 (79G) [application/zip]
Saving to: ‘data_odometry_velodyne.zip’

odyne.zip            97%[==================> ]  77.10G  20.3MB/s    eta 1m 49s 

In [ ]:
# --- 3. UPLOAD VELODYNE DATA TO GCS ---
# The zip extracts into /content/kitti_data/dataset/sequences
upload_dir = '/content/kitti_data/dataset'
upload_to_gcs(upload_dir, 'sentinel-kitti-data', '')

# --- 4. CLEAN UP LOCAL VELODYNE FILES ---
print("\nCleaning up local velodyne files...")
!rm data_odometry_velodyne.zip
!rm -rf /content/kitti_data
print("Cleanup complete.")

In [6]:
import os

# Create a temporary directory for all downloads and move into it
DOWNLOAD_DIR = '/content/temp_kitti'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.chdir(DOWNLOAD_DIR)

# ✅ These are the correct URLs for the complete odometry dataset archives
urls_to_download = {
    "data_odometry_velodyne.zip": "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_velodyne.zip",
    "data_odometry_labels.zip": "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_labels.zip"
}

print("Downloading complete KITTI odometry data archives...")

for filename, url in urls_to_download.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        # Use the !wget shell command with a user-agent to avoid being blocked
        !wget --user-agent="Mozilla/5.0" -O {filename} {url}

        if os.path.exists(filename):
            print(f"Successfully downloaded {filename}")
        else:
            print(f"Error: Failed to download {filename}")
    else:
        print(f"{filename} already exists. Skipping download.")

print("\nAll downloads complete.")

--2025-10-10 04:12:10--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_velodyne.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.134.27, 3.5.138.224, 52.219.170.41, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.134.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84786535790 (79G) [application/zip]
Saving to: ‘data_odometry_velodyne.zip’

data_odometry_velod 100%[===================>]  78.96G  25.6MB/s    in 52m 54s 

2025-10-10 05:05:05 (25.5 MB/s) - ‘data_odometry_velodyne.zip’ saved [84786535790/84786535790]

Successfully downloaded data_odometry_velodyne.zip
--2025-10-10 05:05:05--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_odometry_labels.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.72.227, 3.5.137.34, 3.5.138.164, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.72.227|:443... 

In [12]:
import os

# Define the directory where files will be extracted
EXTRACT_DIR = '/content/kitti_data'
os.makedirs(EXTRACT_DIR, exist_ok=True)

# Define the names of the downloaded files
files_to_unzip = ['data_odometry_velodyne.zip', 'data_odometry_labels.zip']

print("Extracting all zip files using system unzip command...")

for file in files_to_unzip:
    if os.path.exists(file):
        print(f"Extracting {file}...")
        # Use the robust system command !unzip
        # -o: overwrite files without asking
        # -q: quiet mode to prevent printing thousands of filenames
        !unzip -o -q {file} -d {EXTRACT_DIR}
    else:
        print(f"Warning: {file} not found. Skipping extraction.")

print("\nAll files extracted.")

Extracting all zip files using system unzip command...
Extracting data_odometry_velodyne.zip...
Extracting data_odometry_labels.zip...
error [data_odometry_labels.zip]:  missing 16777216 bytes in zipfile
  (attempting to process anyway)
error: invalid zip file with overlapped components (possible zip bomb)

All files extracted.


In [14]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket('sentinel-kitti-data')

# List the first 10 files to verify they are in the 'sequences' folder
blobs = bucket.list_blobs(prefix='sequences/', max_results=10)

print("First 10 files in GCS bucket under 'sequences/':")
for blob in blobs:
    print(f"  - {blob.name}")

# Clean up local temporary files
print("\nCleaning up local files...")
!rm -rf /content/temp_kitti
!rm -rf /content/kitti_data

print("Data setup complete!")

First 10 files in GCS bucket under 'sequences/':

Cleaning up local files...
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Data setup complete!
